In [32]:
import matplotlib
import numpy as np
import pandas as pd
import warnings as wn
from sklearn import metrics
from itertools import cycle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import normalize
from sklearn.cluster import SpectralClusterin, AffinityPropagation, MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.datasets.samples_generator import make_blobs

plt.rcParams['figure.figsize'] = (15, 5)
wn.filterwarnings("ignore")

In [23]:
run = pd.read_csv('results.csv')

In [24]:
run['citizen'] = pd.get_dummies(run.ctz.isna(), drop_first=True)
run['american'] = pd.get_dummies(run.state.isna(), drop_first=True)
run['gender'] = pd.get_dummies(run.gender, drop_first=True)

features = run.drop(columns=['state', 'ctz', 'name', 'city', 'country', 'bib'])

In [38]:
n_clust = 3
X = run.select_dtypes(['number']).drop(columns=['american'])
y = run.select_dtypes(['number']).american
X_train, X_test, y_train, y_test = train_test_split(
  X,y,test_size=0.9,random_state=0)

X_norm = normalize(X)
X_pca = PCA(2).fit_transform(X_norm)
y_pred = KMeans(n_clusters=n_clust, random_state=0).fit_predict(X_pca)
print('Comparing k-means clusters against the data:')
print(pd.crosstab(y_pred, y))

Comparing k-means clusters against the data:
american      0     1
row_0                
0         12657  1630
1          3044   309
2         13707   637


In [26]:
minibatchkmeans = MiniBatchKMeans(init='random',
                                  n_clusters=n_clust,
                                  batch_size=200)
minibatchkmeans.fit(X_pca)
predict_mini = minibatchkmeans.predict(X_pca)
print('Comparing k-means and mini batch k-means solutions:')
print(pd.crosstab(predict_mini, y_pred))

Comparing k-means and mini batch k-means solutions:
col_0      0     1      2
row_0                    
0      14275     0      0
1         12  3353      0
2          0     0  14344


In [30]:
print('Comparing the assigned categories to the ones in the data:')
print(pd.crosstab(y_train,labels))

Comparing the assigned categories to the ones in the data:
col_0       0     1    2
american                
0         699  1647  573
1          90   129   60


In [36]:
n_clusters=n_clust
sc = SpectralClustering(n_clusters=n_clusters)
sc.fit(X_train)
predict=sc.fit_predict(X_train)
print('Comparing the assigned categories to the ones in the data:')
print(pd.crosstab(y_train,predict))

Comparing the assigned categories to the ones in the data:
col_0        0
american      
0         2919
1          279


It looks like the Spectral Clustering is going to be the best.